In [1]:
from sklearn.datasets import fetch_california_housing
import pandas as pd
housing = fetch_california_housing()
housing

{'data': array([[   8.3252    ,   41.        ,    6.98412698, ...,    2.55555556,
           37.88      , -122.23      ],
        [   8.3014    ,   21.        ,    6.23813708, ...,    2.10984183,
           37.86      , -122.22      ],
        [   7.2574    ,   52.        ,    8.28813559, ...,    2.80225989,
           37.85      , -122.24      ],
        ...,
        [   1.7       ,   17.        ,    5.20554273, ...,    2.3256351 ,
           39.43      , -121.22      ],
        [   1.8672    ,   18.        ,    5.32951289, ...,    2.12320917,
           39.43      , -121.32      ],
        [   2.3886    ,   16.        ,    5.25471698, ...,    2.61698113,
           39.37      , -121.24      ]]),
 'target': array([4.526, 3.585, 3.521, ..., 0.923, 0.847, 0.894]),
 'frame': None,
 'target_names': ['MedHouseVal'],
 'feature_names': ['MedInc',
  'HouseAge',
  'AveRooms',
  'AveBedrms',
  'Population',
  'AveOccup',
  'Latitude',
  'Longitude'],
 'DESCR': '.. _california_housing_dataset:\n

In [2]:
data =pd.DataFrame(housing.data, columns=housing.feature_names)
data["price"] = housing.target
data

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,price
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23,4.526
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22,3.585
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24,3.521
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25,3.413
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25,3.422
...,...,...,...,...,...,...,...,...,...
20635,1.5603,25.0,5.045455,1.133333,845.0,2.560606,39.48,-121.09,0.781
20636,2.5568,18.0,6.114035,1.315789,356.0,3.122807,39.49,-121.21,0.771
20637,1.7000,17.0,5.205543,1.120092,1007.0,2.325635,39.43,-121.22,0.923
20638,1.8672,18.0,5.329513,1.171920,741.0,2.123209,39.43,-121.32,0.847


In [3]:
from urllib.parse import urlparse
from sklearn.model_selection import train_test_split
X = data.drop(columns=["price"])
y = data["price"]
X.shape,y.shape

((20640, 8), (20640,))

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33 )
from mlflow.models import infer_signature
signature = infer_signature(X_train,y_train)
X_train.shape,y_train.shape

((13828, 8), (13828,))

In [5]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV

def tuning(X_train, y_train, params):
    rf = RandomForestRegressor()
    grid_search = GridSearchCV(estimator=rf,param_grid=params,cv=5,n_jobs=-1, 
                               scoring="neg_mean_squared_error")
    grid_search.fit(X_train,y_train)
    return grid_search

In [6]:
params = {
    'n_estimators' : [100,200,300],
    'max_depth': [5,10,15,None],
    'min_samples_split': [2,5],
    'min_samples_leaf' : [1,2]
}

In [7]:
import mlflow
from sklearn.metrics import mean_squared_error
with mlflow.start_run():
    grids = tuning(X_train,y_train, params)
    best_model = grids.best_estimator_
    preds = best_model.predict(X_test)
    mse = mean_squared_error(y_test,preds)
    mlflow.log_param("best_n_estimators",grids.best_params_['n_estimators'])
    mlflow.log_param("min_samples_split",grids.best_params_['min_samples_split'])
    mlflow.log_param("best_max_depth",grids.best_params_['max_depth'])
    mlflow.log_param("min_samples_leaf",grids.best_params_['min_samples_leaf'])
    mlflow.log_metric("MSE", mse)
    mlflow.set_tracking_uri(uri='http://127.0.0.1:5000')
    tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

    if tracking_url_type_store != 'file':
        mlflow.sklearn.log_model(best_model, "model",registered_model_name="Best RF Model")
    else:
        mlflow.sklearn.log_model(best_model, "model",signature = signature)

    print(f"Best Hperpaams: {grids.best_params_}")
    print(f"Mean Squared error: {mse}")

KeyboardInterrupt: 